In [31]:
import pandas as pd
import json
import pyperclip

from collections import namedtuple
from decimal import Decimal

In [32]:
with open("cougar_data/stock_value.txt", 'r') as f:
    stock_lines = f.readlines()

In [33]:
def is_stock_line(line):
    if not line.strip() or not line.split():
        return False
    
    s = line.split()[0]
    if '-' not in s or s != s.upper():
        return False
    
    last = s.split('-')[1][-4:]
    return len(s) <= 9 and (last.isdigit() or last.isalpha())

In [34]:
with open("stock_lines.json", 'w') as f:
    json.dump([line.split()[0] for line in stock_lines if is_stock_line(line)], f, indent=4)

In [35]:
StockItem = namedtuple("StockItem", [
    "number", "description", "quantity",
    "location", "price", "total", "value", "equal"
])

In [36]:
i = prev = 0
wrong = []
stock_items = []
while i < len(stock_lines):
    if is_stock_line(stock_lines[i]):
        l = stock_lines[i]
        m = stock_lines[i + 1].replace(".0000", " ", 1)
        m = m.replace('(', '-').replace(')', ' ')
        number = l.split()[0]
        description = l[10:l.index("FIFO")].strip()
        location = m.split()[0]
        quantity = int(m.split()[1])
        price = round(Decimal(m.split()[2]), 2)
        value = round(Decimal(m.split()[3].replace(',', '')), 2)
        equal = "YES"
        if value != quantity * price:
            wrong.append((i, description))
            equal = "NO"

        stock_items.append(StockItem(
            number, description, quantity,
            location, price, quantity * price, value, equal
        ))

        # Safety check
        if 2 < i - prev < 5:
            raise ValueError("Something wrong at line {}".format(i))
        
        prev = i

    i += 2

In [37]:
# convert to dataframe
df = pd.DataFrame(stock_items)
df.head()

,number,description,quantity,location,price,total,value,equal
0,121A-0258,"acc,cash,shawl,hermes,ori",1,HBY,625.00,625.00,625.00,YES
1,126B-0004,"hb,lthr,see-by-chloe,two-",1,HBY,190.00,190.00,190.00,YES
2,172B-0003,"acc,sun-gls,chanel,orig-c",1,HBY,64.00,64.00,64.00,YES
3,1A01-1112,"pin,gp,mosiac,horseshoe,1",1,ABDN,30.00,30.00,30.00,YES
4,1B02-2032,"hb,fur,maurizio-taiuti,2h",1,HBY,65.00,65.00,65.00,YES


In [38]:
# Convert to csv string and copy to clipboard
pyperclip.copy(df.to_csv(sep='\t', index=False))